##### Copyright 2019 The TensorFlow Authors:
https://www.tensorflow.org/tutorials/images/segmentation

Licensed under the Apache License, Version 2.0 (the "License");

#### Traducido para Samsung DesArrolladoras.

# Image segmentation

Este tutorial se enfoca en la tarea de segmentación de imágenes, usando una versión modificada de
 <a href="https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/" class="external">U-Net</a>.

## ¿Qué es la segmentación de imágenes?

Hasta ahora hemos visto la clasificación de imágenes, donde la tarea de la red es asignar una etiqueta o clase a una imagen de entrada. Sin embargo, supongamos que deseamos saber dónde se encuentra un objeto en la imagen, la forma de ese objeto, qué píxel pertenece a qué objeto, etc. En este caso, desearemos segmentar la imagen, es decir, dar una etiqueta a cada píxel de la imagen. Por lo tanto, la tarea de la segmentación de imágenes trata de entrenar una red neuronal para que produzca una máscara de píxeles. Esto ayuda a comprender la imagen a un nivel mucho más bajo, es decir, a nivel de píxeles. La segmentación de imágenes tiene muchas aplicaciones en imágenes médicas, automóviles autónomos e imágenes satelitales, por nombrar algunas.

El dataset que usaremos en este tutorial es el [Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/), creado por Parkhi *et al*.El conjunto de datos consta de imágenes, sus etiquetas correspondientes y máscaras de píxeles. Las máscaras son básicamente etiquetas para cada píxel. Cada píxel tiene una de tres categorías:

* Clase 1: Píxel perteneciente a la mascota.
* Clase 2: Píxel bordeando la mascota.
* Clase 3: Ninguno de los píxeles anteriores / circundantes.

In [0]:
#Puedes correrlo en Colab directamente, o descargar Git primero para correrlo en local.

!pip install git+https://github.com/tensorflow/examples.git
!pip install -U tfds-nightly

In [1]:
import tensorflow as tf

In [2]:
from tensorflow_examples.models.pix2pix import pix2pix

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from IPython.display import clear_output
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow_examples'

## Descarga del dataset Oxford-IIIT Pets 


El conjunto de datos ya está incluido en los conjuntos de datos TensorFlow, todo lo que necesitas hacer es descargarlo. Las máscaras de segmentación se incluyen en la versión 3+.


In [0]:
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

El siguiente código realiza un simple aumento volteando una imagen. Además, la imagen se normaliza a [0,1]. Finalmente, como se mencionó anteriormente, los píxeles en la máscara de segmentación están etiquetados como {1, 2, 3}. Por conveniencia, restamos 1 a la máscara de segmentación, dando como resultado las etiquetas: {0, 1, 2}.

In [0]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

In [0]:
@tf.function
def load_image_train(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  if tf.random.uniform(()) > 0.5:
    input_image = tf.image.flip_left_right(input_image)
    input_mask = tf.image.flip_left_right(input_mask)

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [0]:
def load_image_test(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

El conjunto de datos ya contiene las divisiones requeridas de test y entrenamiento y, por lo tanto, seguimos utilizando la misma división.

In [0]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [0]:
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = dataset['test'].map(load_image_test)

In [0]:
train_dataset = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(BATCH_SIZE)

Echemos un vistazo a un ejemplo de imagen y su máscara correspondiente del conjunto de datos.


In [0]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [0]:
for image, mask in train.take(1):
  sample_image, sample_mask = image, mask
display([sample_image, sample_mask])

## Definimos el modelo

El modelo que se usa aquí es una U-Net modificada. Una U-Net consta de un codificador (muestreador inferior) y un decodificador (muestreador superior). Para aprender características robustas y reducir la cantidad de parámetros entrenables, se puede usar un modelo pre-entrenado como codificador. Por lo tanto, el codificador para esta tarea será un modelo MobileNetV2 previamente entrenado, y se utilizarán sus salidas intermedias, y el decodificador será el bloque de muestreo ascendente ya implementado en los ejemplos de TensorFlow en
 [Pix2pix tutorial](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py). 

La razón para generar tres canales es porque hay tres etiquetas posibles para cada píxel. Piensa en esto como una clasificación múltiple donde cada píxel se clasifica en tres clases.


In [0]:
OUTPUT_CHANNELS = 3

Como se mencionó, el codificador será un modelo MobileNetV2 previamente entrenado que está preparado y listo para usar en
 [tf.keras.applications](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/applications).El codificador consta de salidas específicas de capas intermedias en el modelo. Tenga en cuenta que el codificador no recibirá entrenamiento durante el proceso de training.


In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

# Usamos la activación de estas capas
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Creamos un modelo de extracción de características 
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

El decodificador / upsampler es simplemente una serie de bloques de upsample implementados en ejemplos de TensorFlow.


In [0]:
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [0]:
def unet_model(output_channels):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])
  x = inputs

  # Downsampling a través del modelo
  skips = down_stack(x)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Upsampling y establecer las conexiones de salto
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  # Esta es la última capa del modelo
  last = tf.keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2,
      padding='same')  #64x64 -> 128x128

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

## Entrenamos el modelo

Ahora, todo lo que queda por hacer es compilar y entrenar el modelo. La pérdida que se usa aquí es `losses.SparseCategoricalCrossentropy(from_logits=True)`. La razón para usar esta función de pérdida es porque la red está tratando de asignar una etiqueta a cada píxel, al igual que la predicción multiclase. En la verdadera máscara de segmentación, cada píxel tiene un {0,1,2}. La red aquí está emitiendo tres canales. Esencialmente, cada canal está tratando de aprender a predecir una clase, y `losses.SparseCategoricalCrossentropy(from_logits=True)` es la pérdida recomendada para tal escenario. Usando la salida de la red, la etiqueta asignada al píxel es el canal con el valor más alto. Esto es lo que está haciendo la función create_mask.


In [0]:
model = unet_model(OUTPUT_CHANNELS)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Have a quick look at the resulting model architecture:

In [0]:
tf.keras.utils.plot_model(model, show_shapes=True)

Probemos el modelo para ver qué predice antes del entrenamiento.


In [0]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [0]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [0]:
show_predictions()

Observemos cómo mejora el modelo mientras se entrena. Para realizar esta tarea, a continuación se define una función de callback.


In [0]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [0]:
EPOCHS = 20
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model_history = model.fit(train_dataset, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=test_dataset,
                          callbacks=[DisplayCallback()])

In [0]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs = range(EPOCHS)

plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

## Hacer predicciones

Hagamos algunas predicciones. En aras de ahorrar tiempo, el número de épocas se ha mantenido pequeño, pero puedes configurarlo con más para lograr resultados más precisos.

In [0]:
show_predictions(test_dataset, 3)

## Siguientes pasos

Ahora que comprendes qué es la segmentación de imágenes y cómo funciona, puedes probar este tutorial con diferentes salidas de capa intermedia, o incluso con un modelo previamente entrenado diferente. También puedes desafiarte probando [Carvana](https://www.kaggle.com/c/carvana-image-masking-challenge/overview) un desafío de enmascaramiento de imágenes alojado en Kaggle.

También puedes echar un vistazo a [Tensorflow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection), para otro modelo puedes volver a entrenar con tus propios datos.
